In [ ]:
# Quantum Portfolio Optimization with Qiskit

# !pip install qiskit qiskit-optimization qiskit-aer

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA
from qiskit.primitives import Sampler
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit import Aer
import numpy as np

# -----------------------------
# STEP 1: Define input data
# -----------------------------

# Securities: C = {0, 1, 2}
securities = ['Bond_0', 'Bond_1', 'Bond_2']

# Prices (P_c): in arbitrary units
prices = [100, 200, 150]  # P0, P1, P2

# Max invest (M_c), binary assumption (0/1)
max_invest = [1, 1, 1]

# Portfolio constraints
N = 2  # Max number of bonds to include
target_cashflow = 0.8  # As a fraction of 100MV
MV = 1  # Scaling factor for normalization

# -----------------------------
# STEP 2: Build Qiskit Model
# -----------------------------

# Define a Quadratic Program
qp = QuadraticProgram()

# Add binary decision variables y_c ∈ {0,1}
for i, sec in enumerate(securities):
    qp.binary_var(name=f'y_{i}')

# Objective: Minimize total price of selected bonds
qp.minimize(linear={f'y_{i}': prices[i] for i in range(len(securities))})

# Constraint 1: Limit number of selected bonds to N
qp.linear_constraint(
    linear={f'y_{i}': 1 for i in range(len(securities))},
    sense='==',
    rhs=N,
    name='num_bonds'
)

# Constraint 2: Residual cash flow
# (sum P_c * y_c) / (100 * MV) ≤ target_cashflow
qp.linear_constraint(
    linear={f'y_{i}': prices[i] / (100 * MV) for i in range(len(securities))},
    sense='<=',
    rhs=target_cashflow,
    name='cash_flow'
)

# -----------------------------
# STEP 3: Solve using QAOA
# -----------------------------

# Initialize QAOA sampler
sampler = AerSampler()  # Can use Sampler() if on a real device

qaoa = QAOA(sampler=sampler, reps=1)
optimizer = MinimumEigenOptimizer(min_eigen_solver=qaoa)

# Solve problem
result = optimizer.solve(qp)

# -----------------------------
# STEP 4: Display Result
# -----------------------------

print("🔍 Solution:")
print("Selected Bonds:")
for i in range(len(securities)):
    print(f"  {securities[i]}: {int(result.x[i])}")

print(f"\n💰 Total Cost: {sum(prices[i] * result.x[i] for i in range(len(securities)))}")
